In [1]:
import tensorflow as tf

In [2]:

import numpy as np

In [28]:
data=np.random.random(size=(10,1,2))

In [29]:
data=data[0][0]
data

array([0.35081023, 0.94937476])

In [30]:
def np_to_tf_records(fname,data):
    writer=tf.io.TFRecordWriter(fname)
    feature={}
    feature['data']=tf.train.Feature(float_list=tf.train.FloatList(value=data))
    example=tf.train.Example(features=tf.train.Features(feature=feature))
    serialize=example.SerializeToString()
    writer.write(serialize)
    writer.close()

In [31]:
np_to_tf_records('./myrcrds.tfrecords',data)

In [32]:
data_read=tf.data.TFRecordDataset('./myrcrds.tfrecords')

In [33]:
def parse_func(exp_proto):
    k_to_f={'data':tf.io.FixedLenSequenceFeature([],dtype=tf.float32,allow_missing=True)}
    parsed_feat=tf.io.parse_single_example(serialized=exp_proto,features=k_to_f)
    return parsed_feat['data']

In [34]:
dataset=data_read.map(parse_func)

In [35]:
type(data)

numpy.ndarray

In [36]:
iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)

In [37]:
item=iterator.get_next()

In [38]:
print(item.numpy())

[0.35081023 0.94937474]


In [40]:
import pandas as pd
columns = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
        "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
        "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                    header=None,
                    names=columns)

In [41]:
fname='./pd_data.tfrecords'

In [49]:
Age=tf.train.Feature(int64_list=tf.train.Int64List(value=data['Age']))
WorkClass=tf.train.Feature(bytes_list=tf.train.BytesList(value=[n.encode('utf-8') for n in data['WorkClass']]))
CapitalGain=tf.train.Feature(float_list=tf.train.FloatList(value=data['CapitalGain']))
exp=tf.train.Example(features=tf.train.Features(feature={'Age':Age,'WorkClass':WorkClass,'CapitalGain':CapitalGain}))

In [50]:
writer= tf.io.TFRecordWriter(fname)
writer.write(exp.SerializeToString())
writer.close()

In [52]:
data_from_rcrd=tf.data.TFRecordDataset('./pd_data.tfrecords')

In [64]:
def parse_func_1(exp_proto):
    k_to_f={'Age':tf.io.VarLenFeature(dtype=tf.int64),'WorkClass':tf.io.VarLenFeature(dtype=tf.string),'CapitalGain':tf.io.VarLenFeature(dtype=tf.float32)}
    parsed_feat=tf.io.parse_single_example(serialized=exp_proto,features=k_to_f)
    return parsed_feat['Age'],parsed_feat['WorkClass'],parsed_feat['CapitalGain']

In [65]:
data_frame=data_from_rcrd.map(parse_func_1)


In [66]:
iterator=tf.compat.v1.data.make_one_shot_iterator(data_frame)

In [67]:
item=iterator.get_next()

In [71]:
item[0]